# RATINGS PREDICTION PROJECT

In [1]:
#Importing required libraries

import selenium
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

Scraping Reviews and Ratings from Flipkart

In [13]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
time.sleep(3)

In [14]:
url = "https://www.flipkart.com/"
driver.get(url)
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [15]:
srch_items = ['laptops', 'Phones','smart watches','Headphones','Monitors','professional cameras','printers','Home theater','router']

In [16]:
title = []
review_text = []
ratings = []

def scrape():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [17]:
urls=[]
for i in srch_items:
    
    #Locating search bar
    
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    
    #clicking on search button
    
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:6]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [18]:
len(urls)

576

In [19]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        
        #clicking on all reviews
    
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrape()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrape()
    except: continue

In [20]:
len(ratings), len(review_text), len(title)

(38851, 38851, 38851)

In [21]:
df1 = pd.DataFrame({})

df1["Review_Title"] = title
df1["Review_Text"] = review_text
df1["Rating"] = ratings

df1

,Review_Title,Review_Text,Rating
0,Brilliant,Writing review after 15 day of use.It is absol...,5
1,Delightful,This is thing I want to be...It s worked suprb...,4
2,Terrific,Great product... happy with the purchase,5
3,Nice product,Must buy product .Value for money,4
4,Fabulous!,Bought for basic usage like surfing web and wa...,5
...,...,...,...
38846,Classy product,Emulated HDR I feel of no use.sound is little ...,5
38847,Terrific,Best monitor but delivery it's to late,5
38848,Great product,Excellent monitor. Initially I was little hesi...,5
38849,Really Nice,Nice product,4


In [22]:
df1.to_csv("Flipkart_Ratings_Prediction.csv")